In [ ]:
# This code is Performs PSO on a Discrete Graph
import pickle
import numpy as np
import random

# Loading the database
X = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['x']
print("shape of X:", np.shape(X))

y = pickle.load(open('hydrogen_input_output.pkl', 'rb'))['y']
print("shape of y:", np.shape(y))

# A function to return the index of the input in the database
def input_index(Xinput):
  for i, input in enumerate(X):
    if np.array_equal(Xinput, input):
      return y[i]
  return None

shape of X: (98694, 7)
shape of y: (98694,)


In [ ]:
# The objective function essentially returns the output of the point in the database closest to it
def objective_function(x):
    x = np.array(x)

    # Finding the closest data point in the dataset
    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    return y[closest_index]

# snap_position essentially snaps the input to the point in the database closest to it
def snap_position(x):
    x = np.array(x)

    distances = np.sum((X - x)**2, axis=1)
    closest_index = np.argmin(distances)

    snapped_input = X[closest_index]
    return snapped_input

# We approach Particle Swarm Optimization from a Object-Oriented Perspective, since that's how the algorithm works: a bunch of particles (objects) talking to each other
class Particle:
    def __init__(self, num_dimensions, init_position, index):
        self.position = init_position
        self.velocity = np.zeros(num_dimensions)  # Initially the velocity of each particle is zero
        self.best_position = np.copy(self.position) # The best position seen by the particle initially is its initial position!
        self.best_fitness = objective_function(self.position)
        self.index = index  # the index variable allows us to label and keep track of each particle

# The PSO class actually handles the running of the PSO algorithm
class PSO:
    def __init__(self, num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, init_positions):
        self.num_particles = num_particles  # No. of particles
        self.num_dimensions = num_dimensions  # The dimensions of the dataset
        self.lower_bound = lower_bound  # The lower bound for each dimension of the dataset (we normalize each dimension to lie within [0.0, 1.0])
        self.upper_bound = upper_bound  # Same as above, but upper bound instead
        self.w = w  # w is the weight parameter of each particle; it's a measure of its inertia
        self.c1 = c1  # c1 is a parameter that controls how much the global best position affects its velocity
        self.c2 = c2  # c2 is a parameter that controls how much its personal best position affects its velocity
        self.num_iterations = num_iterations  # Total number of iterations to run PSO for
        self.particles = [Particle(num_dimensions, init_positions[ind], ind) for ind in range(num_particles)] # Initializes a list of particles
        self.global_best_position = np.copy(self.particles[0].position) # global_best_position keeps track of the global best position of all particles. For now, we initialize it arbitrarily
        self.global_best_fitness = self.particles[0].best_fitness # The output of the global_best_position via the database. Again, we initialize it arbitrarily for now

    # This method handles updating each particle's parameters in each iteration
    def update_particles(self):
        for particle in self.particles:
            # Updating the velocity
            particle.velocity = (self.w * particle.velocity
                                 + self.c1 * np.random.rand() * (particle.best_position - particle.position)
                                 + self.c2 * np.random.rand() * (self.global_best_position - particle.position))

            # Updating the position
            particle.position = particle.position + particle.velocity

            # Cliping the position of each particle
            particle.position = np.clip(particle.position, self.lower_bound, self.upper_bound)  # Firstly clipping it so that it stays within the upper and lower bounds
            particle.position = snap_position(particle.position)  # And then snapping it to the closest position within the database

    # This method updates each particle's personal best and the global best
    def update_personal_and_global_bests(self):
        for particle in self.particles:
            # Evaluating the current output of the particle based on its position
            current_output = objective_function(particle.position)

            # Updating the particle's personal best:
            if (current_output > particle.best_fitness):
                particle.best_position = np.copy(particle.position)
                particle.best_fitness = current_output

                # Updating the global best position:
                if (current_output > self.global_best_fitness):
                    self.global_best_position = np.copy(particle.position)
                    self.global_best_fitness = current_output

            # This is mainly for debugging purposes; Feel free to comment out if it clutters the output too much.
            print("Particle #", particle.index, ": ")
            print("-> Current Position: ", particle.position)
            print("-> Current Velocity: ", particle.velocity)
            print("-> Current Output: ", input_index(particle.position))
            print("-> Personal Best Position: ",  particle.best_position)


        print("---> Global Best Position: ", self.global_best_position, " <---")
        print("---> Global Best Output: ", input_index(self.global_best_position), " <---")

    # This method handles the actual PSO process
    def optimize(self):
        for i in range(self.num_iterations):
            print("Iteration #", i, ": ")
            print("x-x-x-x-x-x-x-x-x-x-x")
            self.update_particles()
            self.update_personal_and_global_bests()

        return self.global_best_position, self.global_best_fitness




In [ ]:
# Adjusting Parameters
num_particles = 10
num_dimensions = 7  # Can be automated, but just easier to understand if manually entered
lower_bound = 0
upper_bound = 1.0
# PSO parameters
w = 0.8
c1 = 0.8
c2 = 0.8
num_iterations = 10

# Randomizing initial positions of the particles
initial_positions_PSO = random.choices(X, k=num_particles)
print("Initial Particle Positions: ", initial_positions_PSO)

# Defining the PSO object:
pso = PSO(num_particles, num_dimensions, lower_bound, upper_bound, w, c1, c2, num_iterations, initial_positions_PSO)
print("Beginning PSO Algorithm: ")
best_position, best_fitness = pso.optimize() # Running the optimize method

# Outputting the final results
print("Best position found: ", best_position)
print("Best fitness found: ", best_fitness)

Initial Particle Positions:  [array([0.0407767 , 0.59011795, 0.34406586, 0.90909091, 0.1052337 ,
       0.33029069, 0.19272727]), array([0.43495146, 0.        , 0.        , 0.41414141, 0.00503778,
       0.05687403, 0.03384615]), array([0.21359223, 0.10881333, 0.30086958, 0.56565657, 0.01399384,
       0.08158966, 0.07440559]), array([0.07378641, 0.46809231, 0.467472  , 0.82828283, 0.05597537,
       0.20502738, 0.18111888]), array([0.08932039, 0.48405641, 0.5767881 , 0.81818182, 0.04673943,
       0.11445022, 0.18783217]), array([0.13786408, 0.28070872, 0.50532663, 0.73737374, 0.02742793,
       0.12680803, 0.09972028]), array([0.18446602, 0.14879897, 0.35722474, 0.64646465, 0.018192  ,
       0.07147872, 0.05832168]), array([0.1631068 , 0.15551795, 0.32858409, 0.55555556, 0.01987126,
       0.12849319, 0.12699301]), array([0.09126214, 0.43889128, 0.53609994, 0.84848485, 0.04673943,
       0.15854515, 0.12573427]), array([0.17669903, 0.18008923, 0.41170007, 0.65656566, 0.0193115 ,
   